In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
num_workers = 0
# how many samples per batch to load
batch_size = 32
# percentage of training set to use as validation
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
data_dir='DATASET'
classes=['O','R']

In [ ]:
train_data = datasets.ImageFolder(data_dir + '/TRAIN', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/TEST', transform=test_transforms)

In [ ]:
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [ ]:
model = models.densenet121(pretrained=True)

In [ ]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    

In [ ]:
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

In [ ]:
criterion = nn.NLLLoss()

In [ ]:
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [ ]:
n_epochs = 20
valid_loss_min = np.Inf
max_patience = 3
patience = 0
for epoch in range(1, n_epochs+1):
    train_loss = 0
    model.train()
    for i,(data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        if i % 10 == 0:
            valid_loss = 0
            print(f'Loss at step {i} - {train_loss}')
            model.eval()
            for data, target in valid_loader:
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()*data.size(0)
            valid_loss = valid_loss/len(valid_loader.sampler)
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
                torch.save(model.state_dict(), 'best_model.pt')
                valid_loss_min = valid_loss
            else:
                patience += 1
        if patience >= max_patience:
            print('break training')
            break
    if patience >= max_patience:
            print('break training')
            break
                
    train_loss = train_loss/len(train_loader.sampler)
        
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

Loss at step 0 - 5.99953556060791
Validation loss decreased (inf --> 0.239044).  Saving model ...
Loss at step 10 - 88.82208204269409
Loss at step 20 - 164.84619450569153
Validation loss decreased (0.239044 --> 0.233013).  Saving model ...
Loss at step 30 - 230.32591891288757
Loss at step 40 - 312.5183336734772
break training
break training


In [ ]:
best_dense = torch.load('best_model.pt')

In [ ]:
correct = 0
total = 0
best_dense.eval()
correct_pred, num_examples = 0, 0
for data, target in test_loader:
    output = best_dense(data)
    _, pred = torch.max(output, 1)
    num_examples += target.size(0)
    correct_pred += (pred == target).sum()
test_accuracy = correct_pred.float() / num_examples * 100
print('Test Accuracy: {:.6f}\n'.format(test_accuracy))


Test Accuracy: 95%
